## Проект "Анализ веб-документов" Техносфера Весна 2021

In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from scipy import sparse

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV, train_test_split

import warnings
warnings.filterwarnings("ignore")

Ноутбук работал на каггле, так как там для соревнования бесплатно дается 16 гб оперативной памяти и 4 ядра для вычислений. Также там есть поддержка GPU ускорения, которое могло бы понадобиться для XGBoost

In [2]:
path_to_data = "../input/anomaly-detection-competition-ml1-ts-spring-2021/"
path_to_content = "../input/spheredata/content/content/"
path_to_parsed = "../input/spheredata/data/"

### Посмотрим на наши трейновые данные

In [3]:
traindf = pd.read_csv(path_to_data + 'train_groups.csv', sep=',', index_col='pair_id')
print(traindf.shape)
traindf.head(5)

(11690, 3)


,group_id,doc_id,target
pair_id,,,
1,1,15731,0
2,1,14829,0
3,1,15764,0
4,1,17669,0
5,1,14852,0


# Идея 0 

Посмотрим на каждые заголовки, построим множество признаков, основанных на количестве схожих слов в заголовках.
Будем брать топ 15 в списке

## Решение 0
Повторим решение со второй домашней работы

1. Прочитаем все заголовки

In [4]:
doc_to_title = {}

with open(path_to_data + 'docs_titles.tsv') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_to_title[int(data[0])] = '' if len(data) == 1 else data[1]

filesdf = pd.DataFrame.from_dict(doc_to_title, orient='index', columns=['title'])
filesdf.index.name = 'doc_id'
filesdf.sort_index(inplace=True)
filesdf.head(4)

,title
doc_id,
1,М. Б. Аншина Центр репродукции и генетики «Фер...
2,Переводы Киви кошелька
3,ПРОЕКТ ПАТРУЛИ ВРЕМЕНИ - РЕАБИЛИТАЦИЯ ДУХОВНЫХ...
4,"Блог - Клуб "" Преподавание в начальных класса..."


2. Прочитаем трейн и добавим заголовки к нему

In [5]:
traindf = pd.read_csv(path_to_data + 'train_groups.csv', index_col='pair_id')
traindf.head(4)

,group_id,doc_id,target
pair_id,,,
1,1,15731,0
2,1,14829,0
3,1,15764,0
4,1,17669,0


2. Найдем признаки. Посчитаем общие слова для всех групп и всех веб-страниц в заголовке. Как признаки для веб-страницы, возьмем значения топ-15  пересечений с другими страницами из группы. 

In [6]:
def extract_features(group_df):
    titles = filesdf.loc[group_df['doc_id'].values].title
    matrix = []
    for i, title in enumerate(titles):
        words = set(title.strip().split())
        distances = []
        for j, another_title in enumerate(titles):
            if i == j:
                continue
            another_words = set(another_title.strip().split())
            distances.append(len(words.intersection(another_words)))
        fueatures_to_add = (-np.partition(-np.asarray(distances, dtype='int'), 15)[:15])
        matrix.append(-np.sort(-fueatures_to_add))
    return matrix

In [7]:
def extract_features_for_dataframe(df):
    X = None
    for group_id in df.group_id.unique():
        X = sparse.vstack((X, extract_features(df[df.group_id == group_id])))
    return X

In [8]:
grid = {
    'max_iter': [1000, 2000, 3000, 5000],
    'loss': ['hinge', 'log', 'squared_hinge'],
    'alpha': np.logspace(3, -3, 10),
    'penalty': ['l1', 'l2']
}

In [9]:
X_train = extract_features_for_dataframe(traindf)
y_train = np.asarray(traindf['target'])
groups = np.asarray(traindf['group_id'])

scaler = StandardScaler(with_mean=False).fit(X_train, y_train)
X_train = scaler.transform(X_train);

searcher = GridSearchCV(SGDClassifier(), grid, scoring='f1', n_jobs=-1, pre_dispatch='2*n_jobs')
searcher.fit(X_train, y_train, groups=groups);

best_params = { 'n_jobs': -1 }
best_params.update(searcher.best_params_)
best_params

### Проверим наше предсказание на датасете

In [10]:
X_train_val, X_val, y_train_val, y_val = train_test_split(X_train, y_train, random_state=42)

clf = SGDClassifier(**best_params).fit(X_train_val, y_train_val)
y_pred = clf.predict(X_val)
f1_score(y_pred, y_val)

### Считываем test и делаем predict

In [11]:
testdf = pd.read_csv(path_to_data + 'test_groups.csv', index_col='pair_id')
testdf.head(4)

,group_id,doc_id
pair_id,,
11691,130,6710
11692,130,4030
11693,130,5561
11694,130,4055


In [12]:
X_test = extract_features_for_dataframe(testdf)
X_test = scaler.transform(X_test)

clf = SGDClassifier(**best_params).fit(X_train, y_train)
testdf['target'] = clf.predict(X_test)

testdf.drop(columns=['group_id', 'doc_id']).to_csv('solution0.csv')

### Итоговый скор 0.51914
Надо что-то получше

# Идея 1

Посмотрим на слова в получившихся данных.

In [13]:
from collections import Counter

In [14]:
wordCounter = Counter()
for title in filesdf['title'].values:
    wordCounter.update(title.strip().split())
wordCounter.most_common(5)

[('-', 11915), ('в', 6517), ('и', 5654), ('|', 4754), ('на', 4269)]

Явно среди заголовков оказалось куча мусора и бесполезной информации.
Давайте предобработаем заголовки.

In [15]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 717 kB/s eta 0:00:01
     |████████████████████████████████| 8.2 MB 5.2 MB/s eta 0:00:01
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=f120eb93e6c69d69e68bacc2311940cdffc18ce2ed3321d9dacf26617c354eb3
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [16]:
from nltk import corpus, word_tokenize
from string import digits, punctuation
from pymorphy2 import MorphAnalyzer
import re

In [17]:
emoji_pattern = re.compile(u"[^\U00000000-\U0000d7ff\U0000e000-\U0000ffff]", flags=re.UNICODE)

chars_to_remove = digits + punctuation + "–—«»№•❤★✿╬·…®°º™►"
morph = MorphAnalyzer()
RU_stopwords = set(corpus.stopwords.words('russian'))
EN_stopwords = set(corpus.stopwords.words('english'))
GE_stopwords = set(corpus.stopwords.words('german'))
stopwords = RU_stopwords.union(EN_stopwords.union(GE_stopwords))

In [18]:
def process_text(text):
    if text is None:
        return
    text = emoji_pattern.sub(r'', text)
    text = text.translate(str.maketrans('', '', chars_to_remove))
    words = word_tokenize(text)
    return [morph.parse(word)[0].normal_form for word in words 
             if word not in stopwords and len(word) > 1]

In [19]:
filesdf['title_words'] = filesdf['title'].apply(process_text)
filesdf.drop(columns=['title'], inplace=True)
filesdf.head(5)

,title_words
doc_id,
1,"[аншина, центр, репродукция, генетика, фертиме..."
2,"[перевод, киви, кошелёк]"
3,"[проект, патруль, время, реабилитация, духовны..."
4,"[блог, клуб, преподавание, начальный, класс, п..."
5,"[как, быстро, понизить, холестерин, высокий, х..."


Проверим как изменились наши заголовки

In [20]:
new_wordCounter = Counter()
for words in filesdf['title_words'].values:
    new_wordCounter.update(words)
new_wordCounter.most_common(10)

[('как', 4036),
 ('форум', 1714),
 ('скачать', 1246),
 ('страница', 1040),
 ('онлайн', 949),
 ('бесплатно', 822),
 ('российский', 803),
 ('что', 802),
 ('новость', 738),
 ('гта', 659)]

Уже лучше, попробуем запустить решение 0 на этих данных

## Решение 1

In [21]:
def extract_features(group_df):
    words = filesdf.loc[group_df['doc_id'].values].title_words
    matrix = []
    for i, current_words in enumerate(words):
        distances = []
        for j, another_words in enumerate(words):
            if i == j:
                continue
            distances.append(len(set(current_words) & set(another_words)))
        fueatures_to_add = (-np.partition(-np.asarray(distances, dtype='int'), 15)[:15])
        matrix.append(-np.sort(-fueatures_to_add))
    return matrix

In [22]:
X_train = extract_features_for_dataframe(traindf)
y_train = np.asarray(traindf['target'])
groups = np.asarray(traindf['group_id'])

scaler = StandardScaler(with_mean=False).fit(X_train, y_train)
X_train = scaler.transform(X_train);

searcher = GridSearchCV(SGDClassifier(), grid, scoring='f1', n_jobs=-1, pre_dispatch='2*n_jobs')
searcher.fit(X_train, y_train, groups=groups);

best_params = { 'n_jobs': -1 }
best_params.update(searcher.best_params_)
best_params

### Проверим наше предсказание на датасете

In [23]:
X_train_val, X_val, y_train_val, y_val = train_test_split(X_train, y_train, random_state=42)

clf = SGDClassifier(**best_params).fit(X_train_val, y_train_val)
y_pred = clf.predict(X_val)
f1_score(y_pred, y_val)

### Считываем test и делаем predict

In [24]:
X_test = extract_features_for_dataframe(testdf)
X_test = scaler.transform(X_test)

clf = SGDClassifier(**best_params).fit(X_train, y_train)
testdf['target'] = clf.predict(X_test)

testdf.drop(columns=['group_id', 'doc_id']).to_csv('solution1.csv')

### Итоговый скор 0.65689
Ура, побили бейзлайн! Явно самих заголовков не хватает. Давайте обкачаем наши данные.

# Парсинг

In [25]:
!pip install bs4

     |████████████████████████████████| 115 kB 1.3 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=dfd5b589f297b24bfc9acb6a9481df5df5c4b6395bd339d7b3d1410febd886da
  Stored in directory: /root/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [26]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import codecs

Добавим имя URL без домена и путь.

In [27]:
def process_url(url):
    first_slash = url.find('/')
    last_dot = (url if first_slash == -1 else url[:first_slash]).rfind('.')
    return {
        "url_name": url[:last_dot]
    }

Из головы достанем верификацию от гугла, а также информацию о ключевых словах и описании

In [28]:
def process_head(head):
    if head is None:
        return {
            'verified': 0,
            'auxiliary': []
        }
    verified = int(head.find('meta', attrs={'name': "google-site-verification"}) is not None)

    keywords = head.find('meta', attrs={'name': "keywords", 'content': True})
    keywords = [] if keywords is None else process_text(keywords['content'])

    description = head.find('meta', attrs={'name': "description", "content": True})
    description = [] if description is None else process_text(description['content'])

    return {
        'verified': verified,
        'auxiliary': keywords + description
    }

Из тела возьмем весь читабельный текст

In [29]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def get_visible_text(body):
    texts = body.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return u" ".join(t.strip() for t in visible_texts)

def process_body(body):
    return {
        "text": process_text(get_visible_text(body)) if body is not None else []
    }

Функция обработки страницы

In [30]:
def process_page(file):
    url = file.readline().strip()
    soup = BeautifulSoup(file, 'html.parser')
    page_info = dict()
    page_info.update(process_url(url))
    page_info.update(process_head(soup.find('head')))
    page_info.update(process_body(soup.find('body')))
    return page_info

Посмотрим на какой-нибудь файл

In [31]:
def process_doc(doc_id):
    with codecs.open(path_to_content + f'{doc_id}.dat', 'r', 'utf-8') as f:
        return process_page(f)

In [32]:
def process_docs(doc_ids):
    return {doc_id: process_doc(doc_id) for doc_id in doc_ids}

In [33]:
Counter(process_doc(1)['text']).most_common(10)

[('железа', 59),
 ('щитовидный', 56),
 ('гормон', 35),
 ('определение', 34),
 ('анализ', 33),
 ('кровь', 33),
 ('ошибка', 32),
 ('лабораторный', 31),
 ('гормональный', 30),
 ('уровень', 30)]

Чтобы обкачать все файлы явно одной моей машины не хватит, будем абузить кагл!

## Обкачивание данных

In [34]:
from functools import partial
from tqdm.contrib.concurrent import process_map

In [35]:
def apply_to_array(arr, func, **kwargs):
    func_wrapper = partial(func, **kwargs)
    return process_map(func_wrapper, arr, chunksize=1)

Я разбил дальнейшее обкачивание на 6 кагл ноутбуков, чтобы быстрее все обкачивать и чтобы все влезало в оперативную память, на этот ноутбук выпала третья часть всех данных

In [36]:
chunks = np.array_split(filesdf.index, 6)
current_chunks = chunks[5]
current_chunks[0], current_chunks[-1]

(23356, 28026)

In [37]:
# %%time
# all_data = dict()
# for doc_id, data in enumerate(apply_to_array(current_chunks, extract_data), start=1):
#     all_data.update({doc_id : data})
# len(all_data)

# pd.DataFrame.from_dict(all_data, orient='index').to_csv('data'+str(current_chunks[0])+'to'+str(current_chunks[-1])+'.csv')

Соберем теперь все обкаченные данные в один датафрейм

In [38]:
def get_df(chunk):
    cur_df = pd.read_csv(path_to_parsed + 'data'+str(chunk[0]) + 'to' + str(chunk[-1]) + '.csv', index_col=0)
    cur_df.index = range(chunk[0], chunk[-1] + 1)
    trans = lambda s: s.translate(str.maketrans('', '', "'[,]"))
    cur_df['auxiliary'] = cur_df['auxiliary'].apply(trans)
    cur_df['text'] = cur_df['text'].apply(trans)
    return cur_df

for i, df in enumerate(apply_to_array(chunks, get_df)):
    current_chunk = chunks[i]
    filesdf.loc[current_chunk[0]:current_chunk[-1], df.columns] = df

filesdf.head(4)

  0%|          | 0/6 [00:00<?, ?it/s]

,title_words,url_name,verified,auxiliary,text
doc_id,,,,,
1,"[аншина, центр, репродукция, генетика, фертиме...",zrenielib,1.0,аншина центр репродукция генетика фертимед мос...,аншина центр репродукция генетика фертимед мос...
2,"[перевод, киви, кошелёк]",kak-perevesti-online,0.0,,главный перевод киви кошелёк перевод киви коше...
3,"[проект, патруль, время, реабилитация, духовны...",timecops,0.0,,проект патруль время реабилитация духовный сущ...
4,"[блог, клуб, преподавание, начальный, класс, п...",proffi95,0.0,блог клуб преподавание начальный класс,размер шрифт цвет сайт изображение выклый шриф...


In [39]:
filesdf['title_words'] = filesdf['title_words'].apply(lambda l: " ".join(l))
filesdf.head(4)

,title_words,url_name,verified,auxiliary,text
doc_id,,,,,
1,аншина центр репродукция генетика фертимед москва,zrenielib,1.0,аншина центр репродукция генетика фертимед мос...,аншина центр репродукция генетика фертимед мос...
2,перевод киви кошелёк,kak-perevesti-online,0.0,,главный перевод киви кошелёк перевод киви коше...
3,проект патруль время реабилитация духовный сущ...,timecops,0.0,,проект патруль время реабилитация духовный сущ...
4,блог клуб преподавание начальный класс портал ...,proffi95,0.0,блог клуб преподавание начальный класс,размер шрифт цвет сайт изображение выклый шриф...


Держать слова в листах оказалось не очень хорошей идеей, надо было после исправления сделать джоин, чтобы сохранять строку, так как в итоге все равно все векторайзеры работают с большой строкой :(

# Идея 3

Давайте добавим в качестве признаков не только расстояния между тайтлами, но и косинусовые расстояния между текстами, а также их средние, медианы и все остальное. Для этого для начала добавим в датафрейм данных 60 наиболее встречающихся слов среди всего текста.

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import Counter
from sklearn.metrics import pairwise_distances

In [41]:
def apply_to_subdf(df, attr, func, **kwargs):
    values = getattr(df, attr).unique()
    splitted_dfs = [df[getattr(df, attr) == val] for val in values]
    return apply_to_array(splitted_dfs, func, **kwargs)

In [42]:
def get_max_features(doc_df, max_features):
    data = doc_df.to_dict('records')[0]
    c = Counter()
    c.update(w for w in (data['text'] + ' ' + data['auxiliary']).split() if len(w) > 1)
    return data['url_name'] + ' ' + data['title_words'] + ' ' + " ".join(w[0] for w in c.most_common(max_features))


for i, mfeatures in enumerate(apply_to_subdf(filesdf, 'index', get_max_features, max_features=60), start=1):
    filesdf.at[i, 'max_features'] = mfeatures

  0%|          | 0/28026 [00:00<?, ?it/s]

In [43]:
filesdf.max_features.head(4)

doc_id
1    zrenielib аншина центр репродукция генетика фе...
2    kak-perevesti-online перевод киви кошелёк коше...
3    timecops проект патруль время реабилитация дух...
4    proffi95 блог клуб преподавание начальный клас...
Name: max_features, dtype: object

In [44]:
def extract_features(group_df, n_features):
    datadf = filesdf.loc[group_df['doc_id'].values]

    vec = TfidfVectorizer(sublinear_tf=True)
    tfidf_matrix = vec.fit_transform(datadf.max_features)

    distance_matrix = np.partition(pairwise_distances(tfidf_matrix, metric='cosine', n_jobs=-1), n_features, axis=1)[:, :n_features]
    X = None
    for func in [np.mean, np.average, np.median, np.std, np.var, np.max, np.min]:
        res = func(distance_matrix, axis=0)
        res_column = res[np.newaxis, :]
        X = np.hstack((X if X is not None else distance_matrix, np.repeat(res_column, distance_matrix.shape[0], axis=0)))
    return X


In [45]:
def extract_features_for_dataframe(df):
    X = None
    for x in apply_to_subdf(df, 'group_id', extract_features, n_features=25):
        X = sparse.vstack((X, x))
    return X

In [46]:
X_train = extract_features_for_dataframe(traindf)
y_train = np.asarray(traindf['target'])
groups = np.asarray(traindf['group_id'])

scaler = StandardScaler(with_mean=False).fit(X_train, y_train)
X_train = scaler.transform(X_train);

searcher = GridSearchCV(SGDClassifier(), grid, scoring='f1', n_jobs=-1, pre_dispatch='2*n_jobs')
searcher.fit(X_train, y_train, groups=groups);

best_params = { 'n_jobs': -1 }
best_params.update(searcher.best_params_)
best_params

### Проверим наше предсказание на датасете

In [47]:
X_train_val, X_val, y_train_val, y_val = train_test_split(X_train, y_train, random_state=42)

clf = SGDClassifier(**best_params).fit(X_train_val, y_train_val)
y_pred = clf.predict(X_val)
f1_score(y_pred, y_val)

### Считываем test и делаем predict

In [48]:
X_test = extract_features_for_dataframe(testdf)
X_test = scaler.transform(X_test)

clf = SGDClassifier(**best_params).fit(X_train, y_train)
testdf['target'] = clf.predict(X_test)

testdf.drop(columns=['group_id', 'doc_id']).to_csv('solution2.csv')

### Итоговый скор 0.49922
Хм, улучшения не почувствовалось, думаю потому, что признаки перестали зависеть линейно. Попробуем другой классификатор. Прежде чем это сделать добавим еще два признака: поможем кластеризацией и добавим длину текста как признак.

In [49]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

from sklearn.cluster import KMeans

In [50]:
def extract_features(group_df, tfidf, n_features):
    datadf = filesdf.loc[group_df['doc_id'].values]
    words = datadf.title_words.values
    indices = group_df.index.values - 1
    X = []
    for i, current_words in enumerate(words):
        distances = []
        for j, another_words in enumerate(words):
            if i == j:
                continue
            distances.append(len(set(current_words) & set(another_words)))
        fueatures_to_add = (-np.partition(-np.asarray(distances, dtype='int'), 20)[:20])
        X.append(-np.sort(-fueatures_to_add))
        
    X = np.asarray(X)
    X = np.hstack((X, np.asarray([len(text) for text in datadf.text])[:, np.newaxis]))
    
    svd = TruncatedSVD(n_components=100)
    normalizer = Normalizer(copy=False)
    lsa = make_pipeline(svd, normalizer)
    norm_matrix = lsa.fit_transform(tfidf[indices])

    km = KMeans(n_clusters=2, init='k-means++', max_iter=200)
    km.fit(norm_matrix)
    
    X = np.hstack((X, km.labels_[:, np.newaxis]))

    distance_matrix = np.sort(pairwise_distances(tfidf[indices], metric='cosine', n_jobs=-1), axis=1)[:, 1:n_features]
    X = np.hstack((X, distance_matrix))
    for func in [np.mean, np.average, np.median, np.std, np.max]:
        res = func(distance_matrix, axis=0)
        res_column = res[np.newaxis, :]
        X = np.hstack((X, np.repeat(res_column, distance_matrix.shape[0], axis=0)))
    return X

In [51]:
def extract_features_for_dataframe(df):
    vec = TfidfVectorizer(sublinear_tf=True)
    datadf = filesdf.loc[df['doc_id'].values]
    tfidf_matrix = vec.fit_transform(datadf.max_features)

    X = None
    for x in apply_to_subdf(df, 'group_id', extract_features, tfidf=tfidf_matrix, n_features=25):
        X = sparse.vstack((X, x))
    return X

In [52]:
X_train = extract_features_for_dataframe(traindf)
y_train = np.asarray(traindf['target'])
groups = np.asarray(traindf['group_id'])
X_train.shape, y_train.shape, groups.shape

  0%|          | 0/129 [00:00<?, ?it/s]

((11690, 166), (11690,), (11690,))

In [53]:
scaler = StandardScaler(with_mean=False).fit(X_train, y_train)
X_train_scaled = scaler.transform(X_train)

Посмотрим на скоры следующих классификаторов, прежде чем двигаться дальше. Будем целиться на xgboost stacking с топ 5 лучшими из получившихся.

In [54]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


import matplotlib.pyplot as plt
import seaborn as sns

In [55]:
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3, n_jobs=-1),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0), n_jobs=-1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, n_jobs=-1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [56]:
def collect_scores(classifier):
    clf.fit(X_train_val.A, y_train_val)
    y_pred = clf.predict(X_val.A)
    return f1_score(y_pred, y_val)

scores = []
for score in apply_to_array(classifiers, collect_scores):
    scores.append(score)

In [57]:
fig, ax = plt.subplots(figsize=(12, 5))
ax.set(xlabel='Highest score')
sns.barplot(x=names, y=scores, saturation=0.3, orient='v');

Подберем сначала параметры для xgboosta

In [58]:
import xgboost as xgb
from sklearn.model_selection import GroupKFold

In [127]:
def f1_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = 1-f1_score(y_true, np.round(y_pred))
    return 'f1_err', err

In [128]:
opt_params = {
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'verbosity': 0,
    'eval_metrix': f1_eval
}

In [169]:
def ValScore(X, y, groups, n_splits=5, *args, **kwargs):
    kf = GroupKFold(n_splits=n_splits)
    clf = (*args, **kwargs)

    scores = []
    for train, test in kf.split(X, y, groups=groups):
        X_train, y_train = X.A[train], y[train]
        X_test, y_test = X.A[test], y[test]
        clf.fit(X_train, y_train)
        scores.append(f1_score(y_pred=clf.predict(X_test),
                               y_true=y_test))
    return np.asarray(scores)

In [170]:
def FindParams(X, y, groups, param_name, param_range, known_params=opt_params):
    mean_scores = []

    for param in param_range:
        kwargs = known_params
        kwargs.update({param_name: param})
        scores = ValScore(X, y, groups, **kwargs)
        mean_scores.append(scores.mean())

    opt_param = param_range[np.argmax(mean_scores)]

    plt.figure(figsize=(10, 6))
    plt.xlabel(param_name)
    plt.ylabel('score')
    plt.title(f'Зависимость f1-score от параметра {param_name}\n'
              f'Оптимальное значение параметра {param_name}: {opt_param}')
    plt.plot(param_range, mean_scores)

    return opt_param

In [171]:
grid = {
    'objective': ['binary:logistic', 'binary:hinge'],
    'n_estimators': np.linspace(20, 200, 30).astype(int),
    'booster': ['gbtree'],
    'eta': np.arange(0.05, 0.3, 0.05),
    'gamma': np.logspace(2, -3, 5),
    'max_depth': np.arange(3, 10, 2).astype(int),
    'min_child_weigth': np.logspace(0, -4, 5),
    'subsampe': np.arange(0.4, 1, 0.1),
    'alpha': np.logspace(1, -4, 5),
    'colsample_bytree': np.arange(0.5, 1, 0.1)
}

In [172]:
for param_name in grid:
    range_ = grid[param_name]
    opt_param = FindParams(X_train_scaled, y_train, groups, param_name, range_, opt_params)
    opt_params.update({param_name: opt_param})

objective


TypeError: 'numpy.float64' object cannot be interpreted as an integer

Из-за этой ошибки я так и не смог получил скор, который возможно мог бы поднять меня вышел в лидерборде

In [ ]:
opt_params

In [ ]:
X_train_val, X_val, y_train_val, y_val = train_test_split(X_train, y_train, random_state=42)

clf = xgb.XGBClassifier(**opt_params).fit(X_train_val, y_train_val)
y_pred = clf.predict(X_val)
f1_score(y_pred, y_val)

In [ ]:
X_test = extract_features_for_dataframe(testdf)
X_test = scaler.transform(X_test)

clf = xgb.XGBClassifier(**best_params).fit(X_train, y_train)
testdf['target'] = clf.predict(X_test)

testdf.drop(columns=['group_id', 'doc_id']).to_csv('solution3.csv')

Дальше жалкие попытки попробовать другие алгоритмы(

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
opt_params = {
    'n_jobs': -1,
}

In [ ]:
grid = {
    'criterion': ['gini'],
    'n_estimators': [100, 200],
    'max_depth': [6],
    'min_samples_split': [1, 2, 3],
    'min_samples_leaf': [1, 2, 3]
}

In [ ]:
searcher = GridSearchCV(RandomForestClassifier(**opt_params), grid, n_jobs=-1,  scoring='f1', pre_dispatch='2*n_jobs', verbose=1)
searcher.fit(X_train, y_train, groups=groups);
opt_params.update(searcher.best_params_)

In [ ]:
X_train_val, X_val, y_train_val, y_val = train_test_split(X_train, y_train, random_state=42)

clf = xgb.RandomForestClassifier(**opt_params).fit(X_train_val, y_train_val)
y_pred = clf.predict(X_val)
f1_score(y_pred, y_val)

In [ ]:
X_test = extract_features_for_dataframe(testdf)
X_test_scaled = scaler.transform(X_test)

In [ ]:
clf = RandomForestClassifier(**opt_params).fit(X_train_scaled, y_train)
testdf['target'] = clf.predict(X_test_scaled)

testdf.drop(columns=['group_id', 'doc_id']).to_csv('solution3.csv')

0.59047

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
opt_params = {
    'max_depth': 6,
    'reg_alpha': 0.01,
    'objective': 'binary',
    'max_bin': 5
}

In [ ]:
clf = LGBMClassifier(**opt_params).fit(X_tr_scaled, y_train)
testdf['target'] = clf.predict(X_test_scaled)

testdf.drop(columns=['group_id', 'doc_id']).to_csv('solution4.csv')

0.61313

Что-то пошло не так, и я к сожалению не совсем понял где. Есть подозрение, что признаки по косинусовым расстояниям выбирались неправильно. Я брал слова часто встречаемые во всех документах, при этом совсем не обращал внимания на слова в документах, помеченных 1. Это должно быть важно, так как есть группы в которых правильных документов намного меньше, чем неправильных, что и сыграло злую шутку. 

Также думаю, что подход был изначально неправильный, стоило сначала попробовать все алгоритмы на первых данных, затем только добавлять новые фичи и смотреть улучшился ли скор или нет.

В качестве признаков можно было бы еще брать количество картинок в документе, доступен ли документ на данный момент и есть ли у данного документа редирект на какой-нибудь сайт. 

Топ2 в соревновании заняли алгоритмы CatBoost и XGBoost, некоторые использовали DBSCAN для определения выбросов.